Steps:
- Scarping Job postings with Selenium and bs4
- Visualize in Tableau or PowerBI

import necessary modules - first try with BeautifulSoup

In [23]:
import driver as driver
import pandas as pd
import re
import lxml
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
import datetime
start_time = time()

from warnings import warn


Select wanted url

In [24]:
city = str(input('Please enter the city you want to work in'))
state = str(input('Please enter the state you want to work in'))
country = str(input('Please enter the country you want to work in'))
url = 'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location='+ city + '%2C%20'+ state +'%2C%20'+ country

no_of_jobs = 51

url



'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Zuirch%2C%20Zurich%2C%20Switzerland'

open windows with url stated above in

In [25]:
driver = webdriver.Chrome()
driver.get(url)
sleep(3)
action = ActionChains(driver)

to show more jobs than the initial 25

In [ ]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
sleep(20)



parsing the visible webpage

In [ ]:
pageSource = driver.page_source
soup = BeautifulSoup(pageSource,  "html.parser")



Parsing content

In [ ]:
first_job = soup.find('h3', class_ = 'result-card__title job-result-card__title').text
print(first_job)

first_company = soup.find('a', class_ = 'result-card__subtitle-link job-result-card__subtitle-link').text
print(first_company)

Create job container

In [ ]:
job_container = soup.find('ul', class_="jobs-search__results-list")
print(len(job_container))

create for loop to look for all jobs

In [ ]:
print('You are scraping information about {} jobs.'.format(len(job_container)))

In [ ]:
job_titles = []
companies = []
locations = []
time = []
date = []


for jobs in job_container:
    post_title = jobs.find('h3', class_ = 'result-card__title job-result-card__title').text
    job_titles.append(post_title)

    post_company = jobs.find('h4', class_ = 'result-card__subtitle job-result-card__subtitle').text
    companies.append(post_company)

    post_locations = jobs.find('span', class_ = 'job-result-card__location').text
    locations.append(post_locations)

    post_date = jobs.select_one('time')['datetime']
    date.append(post_date)


In [ ]:
ml = {'Title': job_titles, 'Company': companies, 'Locations': locations, 'Post Date': date}
print(ml)

In [ ]:
df = pd.DataFrame(ml)
df.head(1)

Separate City from State and Country


In [ ]:
new = df.Locations.str.split(',', n = 0, expand = True)

In [ ]:
new2 = new.rename(columns = {0: 'City2'})

In [ ]:
new2


merge df and new, drop Locations as it is redundant now

In [ ]:
df2 = df.join(new2, how = 'left')
df2.drop(columns = ['Locations',1 , 2], inplace = True)
df2.head(4)

In [ ]:
new3 = df2.City2.str.split(' ', expand=True)
new4 = new3.rename(columns = {0: 'City'})
new5 = new4.City
new5.head()

In [ ]:
df4 = df2.join(new5, how = 'left')
df4.drop(columns = ["City2"], inplace = True)
df4.set_index('Title', inplace = True)
df4

create CSV, with todays Date

In [ ]:
date_object = datetime.date.today()
date = str(date_object) + '_'
date

df4.to_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/' + date + city + '_' + country + '.csv')

concat new jobs to existing list

In [ ]:
final_df = pd.read_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/01_Final DF/Final_DF_LinkedIN Jobs.csv')
final_df.set_index('Title', inplace = True)
final_df.head()

In [ ]:
dfWIP = pd.concat([final_df, df4], axis=0)
dfWIP.head()

drop duplicated rows

In [ ]:
new_final_df = dfWIP.drop_duplicates()
new_final_df

In [ ]:
new_final_df.to_csv('C:/Users/Marc/Dropbox/01_Marc Arnold/Python Projects/04_Job Viz/02_Output/01_Final DF/Final_DF_LinkedIN Jobs.csv')

